In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score as cvs
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score as f1
from sklearn.metrics import confusion_matrix, precision_score, recall_score 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC 
import pandas as pd
from pandas import ExcelFile
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import scipy.io as io
from sklearn.model_selection import GridSearchCV as gs
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

In [2]:
def show_class(output):
    if output == 1:
        print("Mild asthma")
    elif output == 2:
        print("Moderate asthma")
    else:
        print("Severe asthma")

In [3]:
xl = pd.ExcelFile("sarp.xlsx")
ds = xl.parse('sarp')
X_uncleaned = ds.loc[:, "Baseline_preDrug_FEV1pp":"Baseline_preDrug_FEV1_FVC"]
imputer = Imputer(strategy="median")
imputer.fit(X_uncleaned)
X_cleaned = imputer.transform(X_uncleaned)
X = pd.DataFrame(X_cleaned, columns=X_uncleaned.columns).astype(np.float64)
y = ds.loc[:,"ageasthonset"].astype(np.float64)
stds = ds['status_factorized'].astype(np.int64)
X2 = X
X2 = X["Baseline_preDrug_FEV1_FVC"]
X2 = pd.DataFrame(X2)
X2 = X2.join(y)
X2 = X2.join(ds["gender"])
y2 = stds
y2 = y2.values.ravel()
y2pd = pd.DataFrame(y2)
X2pd = X2
X2 = pd.DataFrame.as_matrix(X2)
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.33)
print(X_train.shape)
print(y_train.shape)

(909, 3)
(909,)


In [3]:
svm_clf = Pipeline([("scaler", StandardScaler()),
                   ("linear_svc", LinearSVC(C=1, loss="hinge"))])
polynomial_svm_clf = Pipeline([("poly_features", PolynomialFeatures(degree=3)), 
                               ("scaler", StandardScaler()), 
                               ("poly_clf", LinearSVC(C=10, loss="hinge"))])
kernel_svm_clf = Pipeline([("scaler", StandardScaler()), 
                                ("kernel_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))]) 

In [19]:
svm_clf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [20]:
res = svm_clf.predict([[.65, 45, 2]]).astype(np.int32)
show_class(res[0])

Severe asthma


In [21]:
scores = cvs(svm_clf, X2, y2, scoring="neg_mean_squared_error", cv=2)
svm_scores = np.sqrt(-scores)
print(svm_scores)

[ 0.98965235  1.07052216]


In [22]:
y_pred = svm_clf.predict(X_test).astype(np.int32)

In [29]:
print(confusion_matrix(y_test, y_pred))
print(f1(y_test,y_pred,average='micro'))
print(precision_score(y_test, y_pred, average='micro'))
print(recall_score(y_test, y_pred, average='micro'))

[[ 85   0  76]
 [ 49   0  40]
 [107   0  92]]
0.39420935412
0.39420935412
0.39420935412


In [ ]:
#param_grid = [ {'kernel_clf__kernel': ["poly", "rbf"], 'kernel_clf__degree': [3, 4, 5, 6], 
#                'kernel_clf__coef0': [0.5, 1, 1.5], 'kernel_clf__C': [1, 2, 3, 4, 5]} ]
#grid_search = gs(kernel_svm_clf, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

In [ ]:
ada_clf = AdaBoostClassifier(svm_clf)

In [ ]:
ada_clf.fit(X_train, y_train)